Deep Learning
=============

Assignment 6
------------

After training a skip-gram model in `5_word2vec.ipynb`, the goal of this notebook is to train a LSTM character model over [Text8](http://mattmahoney.net/dc/textdata) data.

In [1]:
# These are all the modules we'll be using later. Make sure you can import them
# before proceeding further.
from __future__ import print_function
import os
import numpy as np
import random
import string
import tensorflow as tf
import zipfile
from six.moves import range
from six.moves.urllib.request import urlretrieve

In [2]:
url = 'http://mattmahoney.net/dc/'

def maybe_download(filename, expected_bytes):
  """Download a file if not present, and make sure it's the right size."""
  if not os.path.exists(filename):
    filename, _ = urlretrieve(url + filename, filename)
  statinfo = os.stat(filename)
  if statinfo.st_size == expected_bytes:
    print('Found and verified %s' % filename)
  else:
    print(statinfo.st_size)
    raise Exception(
      'Failed to verify ' + filename + '. Can you get to it with a browser?')
  return filename

filename = maybe_download('text8.zip', 31344016)

Found and verified text8.zip


In [3]:
def read_data(filename):
  with zipfile.ZipFile(filename) as f:
    name = f.namelist()[0]
    data = tf.compat.as_str(f.read(name))
  return data
  
text = read_data(filename)
print('Data size %d' % len(text))

Data size 100000000


Create a small validation set.

In [4]:
valid_size = 1000
valid_text = text[:valid_size]
train_text = text[valid_size:]
train_size = len(train_text)
print(train_size, train_text[:64])
print(valid_size, valid_text[:64])

99999000 ons anarchists advocate social relations based upon voluntary as
1000  anarchism originated as a term of abuse first used against earl


Utility functions to map characters to vocabulary IDs and back.

In [5]:
vocabulary_size = len(string.ascii_lowercase) + 1 # [a-z] + ' '
first_letter = ord(string.ascii_lowercase[0]) # ord return the unicode id of a letter.

def char2id(char):
  if char in string.ascii_lowercase:
    return ord(char) - first_letter + 1
  elif char == ' ':
    return 0
  else:
    print('Unexpected character: %s' % char)
    return 0
  
def id2char(dictid):
  if dictid > 0:
    return chr(dictid + first_letter - 1)
  else:
    return ' '

print(char2id('a'), char2id('z'), char2id(' '), char2id('ï'))
print(id2char(1), id2char(26), id2char(0))

Unexpected character: ï
1 26 0 0
a z  


Function to generate a training batch for the LSTM model.

In [6]:
batch_size=64
num_unrollings=10

class BatchGenerator(object):
  def __init__(self, text, batch_size, num_unrollings):
    self._text = text
    self._text_size = len(text)
    self._batch_size = batch_size
    self._num_unrollings = num_unrollings
    segment = self._text_size // batch_size
    self._cursor = [ offset * segment for offset in range(batch_size)]
    self._last_batch = self._next_batch()
  
  def _next_batch(self):
    """Generate a single batch from the current cursor position in the data."""
    batch = np.zeros(shape=(self._batch_size, vocabulary_size), dtype=np.float)
    for b in range(self._batch_size):
      batch[b, char2id(self._text[self._cursor[b]])] = 1.0
      self._cursor[b] = (self._cursor[b] + 1) % self._text_size
    return batch
  
  def next(self):
    """Generate the next array of batches from the data. The array consists of
    the last batch of the previous array, followed by num_unrollings new ones.
    """
    batches = [self._last_batch]
    for step in range(self._num_unrollings):      
      batches.append(self._next_batch())    
      
    self._last_batch = batches[-1]
    return batches

def characters(probabilities):
  """Turn a 1-hot encoding or a probability distribution over the possible
  characters back into its (most likely) character representation."""
  return [id2char(c) for c in np.argmax(probabilities, 1)]

def batches2string(batches):
  """Convert a sequence of batches back into their (most likely) string
  representation."""
  s = [''] * batches[0].shape[0]
  for b in batches:
    s = [''.join(x) for x in zip(s, characters(b))]
  return s

train_batches = BatchGenerator(train_text, batch_size, num_unrollings)
valid_batches = BatchGenerator(valid_text, 1, 1)

print(batches2string(train_batches.next()))
print(batches2string(train_batches.next()))
print(batches2string(valid_batches.next()))
print(batches2string(valid_batches.next()))

['ons anarchi', 'when milita', 'lleria arch', ' abbeys and', 'married urr', 'hel and ric', 'y and litur', 'ay opened f', 'tion from t', 'migration t', 'new york ot', 'he boeing s', 'e listed wi', 'eber has pr', 'o be made t', 'yer who rec', 'ore signifi', 'a fierce cr', ' two six ei', 'aristotle s', 'ity can be ', ' and intrac', 'tion of the', 'dy to pass ', 'f certain d', 'at it will ', 'e convince ', 'ent told hi', 'ampaign and', 'rver side s', 'ious texts ', 'o capitaliz', 'a duplicate', 'gh ann es d', 'ine january', 'ross zero t', 'cal theorie', 'ast instanc', ' dimensiona', 'most holy m', 't s support', 'u is still ', 'e oscillati', 'o eight sub', 'of italy la', 's the tower', 'klahoma pre', 'erprise lin', 'ws becomes ', 'et in a naz', 'the fabian ', 'etchy to re', ' sharman ne', 'ised empero', 'ting in pol', 'd neo latin', 'th risky ri', 'encyclopedi', 'fense the a', 'duating fro', 'treet grid ', 'ations more', 'appeal of d', 'si have mad']
['ists advoca', 'ary governm', 'hes nat

In [7]:
def logprob(predictions, labels):
  """Log-probability of the true labels in a predicted batch."""
  predictions[predictions < 1e-10] = 1e-10
  return np.sum(np.multiply(labels, -np.log(predictions))) / labels.shape[0]

def sample_distribution(distribution):
  """Sample one element from a distribution assumed to be an array of normalized
  probabilities.
  """
  r = random.uniform(0, 1)
  s = 0
  for i in range(len(distribution)):
    s += distribution[i]
    if s >= r:
      return i
  return len(distribution) - 1

def sample(prediction):
  """Turn a (column) prediction into 1-hot encoded samples."""
  p = np.zeros(shape=[1, vocabulary_size], dtype=np.float)
  p[0, sample_distribution(prediction[0])] = 1.0
  return p

def random_distribution():
  """Generate a random column of probabilities."""
  b = np.random.uniform(0.0, 1.0, size=[1, vocabulary_size])
  return b/np.sum(b, 1)[:,None]

Simple LSTM Model.

What this model does was not clear at all, so here are some temptative at understanding it.

We want to create some sort of language model in which we try to predict the next character in a sentence given the previous chars of the same sentence. We try to predict one single char for the moment because that makes the vocabulary size (and the number of possible outcomes of the LSTM predictor) reasonable.

The output of the character prediction model may depend on inputs that are quite distants (think about which gender to use in a sentence or whether some word should be singular or plural). This can make training quite complicated. Instead of using very long sentences as inputs, we will only consider fixed length sequences of 10 characters (parameter `num_unrollings`). This means that we assume that the next char in a sentence can't be influenced by a character that appeared more than 10 chars ago. 

The optimization of the parameters is made by stochastic gradient descent using batches of inputs of length 64 (i.e. each batch is made of 64 sequences of 10 characters). The loss function is the average multiclass cross-entropy between the predicted class probabilities and the true class : 
$$
H = -\frac{1}{N}\sum_{i=1}^N \sum_{j\in \{\alpha\}}y_{ij}\log(p_{ij})
$$
Where $y_{ij}$ is the true label of the $i^{th}$ input, one-hot-encoded, and $p_{ij}$ is the class probability predicted for the same $i^{th}$ input.

In tensorflow, class probabilities are computed using the softmax function applied to a linear classifier. However, the cross entropy is computed from the output (which are in the form of logits, i.e. varying from $-\infty$ to $+\infty$) in one operation. 

The quality of the LSTM is evaluated using a slightly different metric, perplexity, which is roughly an exponential function of the cross-entropy.

In [10]:
num_nodes = 64

graph = tf.Graph()
with graph.as_default():
  
  # Parameters:
  # Input gate: input, previous output, and bias.
  ix = tf.Variable(tf.truncated_normal([vocabulary_size, num_nodes], -0.1, 0.1))
  im = tf.Variable(tf.truncated_normal([num_nodes, num_nodes], -0.1, 0.1))
  ib = tf.Variable(tf.zeros([1, num_nodes]))
  # Forget gate: input, previous output, and bias.
  fx = tf.Variable(tf.truncated_normal([vocabulary_size, num_nodes], -0.1, 0.1))
  fm = tf.Variable(tf.truncated_normal([num_nodes, num_nodes], -0.1, 0.1))
  fb = tf.Variable(tf.zeros([1, num_nodes]))
  # Memory cell: input, state and bias.                             
  cx = tf.Variable(tf.truncated_normal([vocabulary_size, num_nodes], -0.1, 0.1))
  cm = tf.Variable(tf.truncated_normal([num_nodes, num_nodes], -0.1, 0.1))
  cb = tf.Variable(tf.zeros([1, num_nodes]))
  # Output gate: input, previous output, and bias.
  ox = tf.Variable(tf.truncated_normal([vocabulary_size, num_nodes], -0.1, 0.1))
  om = tf.Variable(tf.truncated_normal([num_nodes, num_nodes], -0.1, 0.1))
  ob = tf.Variable(tf.zeros([1, num_nodes]))
  # Variables saving state across unrollings.
  saved_output = tf.Variable(tf.zeros([batch_size, num_nodes]), trainable=False)
  saved_state = tf.Variable(tf.zeros([batch_size, num_nodes]), trainable=False)
  # Classifier weights and biases.
  w = tf.Variable(tf.truncated_normal([num_nodes, vocabulary_size], -0.1, 0.1))
  b = tf.Variable(tf.zeros([vocabulary_size]))
  
  # Definition of the cell computation.
  def lstm_cell(i, o, state):
    """Create a LSTM cell. See e.g.: http://arxiv.org/pdf/1402.1128v1.pdf
    Note that in this formulation, we omit the various connections between the
    previous state and the gates."""
    input_gate = tf.sigmoid(tf.matmul(i, ix) + tf.matmul(o, im) + ib)
    forget_gate = tf.sigmoid(tf.matmul(i, fx) + tf.matmul(o, fm) + fb)
    update = tf.matmul(i, cx) + tf.matmul(o, cm) + cb
    state = forget_gate * state + input_gate * tf.tanh(update)
    output_gate = tf.sigmoid(tf.matmul(i, ox) + tf.matmul(o, om) + ob)
    return output_gate * tf.tanh(state), state

  # Input data.
  train_data = list()
  for _ in range(num_unrollings + 1):
    train_data.append(
      tf.placeholder(tf.float32, shape=[batch_size,vocabulary_size]))
  train_inputs = train_data[:num_unrollings]
  train_labels = train_data[1:]  # labels are inputs shifted by one time step.

  # Unrolled LSTM loop.
  outputs = list()
  output = saved_output
  state = saved_state
  for i in train_inputs:
    output, state = lstm_cell(i, output, state)
    outputs.append(output)

  # State saving across unrollings.
  with tf.control_dependencies([saved_output.assign(output),
                                saved_state.assign(state)]):
    # Classifier.
    logits = tf.nn.xw_plus_b(tf.concat(outputs, 0), w, b)
    loss = tf.reduce_mean(
      tf.nn.softmax_cross_entropy_with_logits(
        labels=tf.concat(train_labels, 0), logits=logits))

  # Optimizer.
  global_step = tf.Variable(0)
  learning_rate = tf.train.exponential_decay(
    10.0, global_step, 5000, 0.1, staircase=True)
  optimizer = tf.train.GradientDescentOptimizer(learning_rate)
  gradients, v = zip(*optimizer.compute_gradients(loss))
  gradients, _ = tf.clip_by_global_norm(gradients, 1.25)
  optimizer = optimizer.apply_gradients(
    zip(gradients, v), global_step=global_step)

  # Predictions.
  train_prediction = tf.nn.softmax(logits)
  
  # Sampling and validation eval: batch 1, no unrolling.
  sample_input = tf.placeholder(tf.float32, shape=[1, vocabulary_size])
  saved_sample_output = tf.Variable(tf.zeros([1, num_nodes]))
  saved_sample_state = tf.Variable(tf.zeros([1, num_nodes]))
  reset_sample_state = tf.group(
    saved_sample_output.assign(tf.zeros([1, num_nodes])),
    saved_sample_state.assign(tf.zeros([1, num_nodes])))
  sample_output, sample_state = lstm_cell(
    sample_input, saved_sample_output, saved_sample_state)
  with tf.control_dependencies([saved_sample_output.assign(sample_output),
                                saved_sample_state.assign(sample_state)]):
    sample_prediction = tf.nn.softmax(tf.nn.xw_plus_b(sample_output, w, b))

In [11]:
num_steps = 10001
summary_frequency = 100

with tf.Session(graph=graph) as session:
  tf.global_variables_initializer().run()
  print('Initialized')
  mean_loss = 0
  for step in range(num_steps):
    batches = train_batches.next()
    feed_dict = dict()
    for i in range(num_unrollings + 1):
      feed_dict[train_data[i]] = batches[i]
    _, l, predictions, lr = session.run(
      [optimizer, loss, train_prediction, learning_rate], feed_dict=feed_dict)
    mean_loss += l
    if step % summary_frequency == 0:
      if step > 0:
        mean_loss = mean_loss / summary_frequency
      # The mean loss is an estimate of the loss over the last few batches.
      print(
        'Average loss at step %d: %f learning rate: %f' % (step, mean_loss, lr))
      mean_loss = 0
      labels = np.concatenate(list(batches)[1:])
      print('Minibatch perplexity: %.2f' % float(
        np.exp(logprob(predictions, labels))))
      if step % (summary_frequency * 10) == 0:
        # Generate some samples.
        print('=' * 80)
        for _ in range(5):
          feed = sample(random_distribution())
          sentence = characters(feed)[0]
          reset_sample_state.run()
          for _ in range(79):
            prediction = sample_prediction.eval({sample_input: feed})
            feed = sample(prediction)
            sentence += characters(feed)[0]
          print(sentence)
        print('=' * 80)
      # Measure validation set perplexity.
      reset_sample_state.run()
      valid_logprob = 0
      for _ in range(valid_size):
        b = valid_batches.next()
        predictions = sample_prediction.eval({sample_input: b[0]})
        valid_logprob = valid_logprob + logprob(predictions, b[1])
      print('Validation set perplexity: %.2f' % float(np.exp(
        valid_logprob / valid_size)))

Initialized
Average loss at step 0: 3.295368 learning rate: 10.000000
Minibatch perplexity: 26.99
b ue  r h eqlr  rt aysoaonhuyqhaosnn  k  cj qv altqkowm m a cnkl lyde zspqeitjaa
yxtijn caglllmw qjs nciqdfheoeky jmh uxt polfo   gp jkmerk v fa nd rthccye a  o 
vxjn otkux zuesbti ytao lbsmivn ieq nifp udwesinckeka  g nm dhwnckcbngpthwalgwig
kelvspcnk qsi njszrtpamcablfnpmnmwsdnye oshbdfmlf h ee  kh  gwfcty eonenrnnlnoji
jvsj js  hh ua e fw pwwth zzuxsedndimylwuiedl j e  dty ggoscmnseuwcp gyf knrow  
Validation set perplexity: 20.15
Average loss at step 100: 2.596106 learning rate: 10.000000
Minibatch perplexity: 10.67
Validation set perplexity: 10.03
Average loss at step 200: 2.247379 learning rate: 10.000000
Minibatch perplexity: 8.47
Validation set perplexity: 8.50
Average loss at step 300: 2.111582 learning rate: 10.000000
Minibatch perplexity: 7.50
Validation set perplexity: 8.11
Average loss at step 400: 2.013580 learning rate: 10.000000
Minibatch perplexity: 7.49
Validation set per

---
Problem 1
---------

You might have noticed that the definition of the LSTM cell involves 4 matrix multiplications with the input, and 4 matrix multiplications with the output. Simplify the expression by using a single matrix multiply for each, and variables that are 4 times larger.

---

The LSTM behaves according to the following equations :
$$
\begin{align}
i_t&=\sigma(W_{ix}x_t+W_{im}o_{t-1}+b_i)\\
f_t&=\sigma(W_{fx}x_t+W_{fm}o_{t-1}+b_f)\\
c_t&=f_t\odot c_{t-1}+i_t\odot\tanh(W_{cx}x_t+W_{cm}o_{t-1}+b_c)\\
o_t&= \sigma(W_{ox}x_t+W_{om}o_{t-1}+b_o)\\
y_t&=w(o_t\odot\tanh(c_t))+b
\end{align}\\
$$

where :
- $x_t$ is the input to the LSTM cell.
- $o_t$ is the output to the LSTM cell.
- $f_t$ is the output of the forget gate.
- $c_t$ is the state of the cell.
- $o_t$ is the output of the output gate.
- $y_t$ is the output of the classifier.
- $\odot$ denotes the Hadamard (element-wise) matrix product.

The first four equations contains the same matrix products :
$$
a_t \sim W_{ax}x_t+W_{am}o_{t-1}+b_a.
$$
where the matrices have the following dimensions :
- $W_{ax}$ : vocabulary_size x num_nodes
- $x_{t}$ : vocabulary_size x 1
- $W_{am}$ : num_nodes x num_nodes
- $o_{t-1}$ : num_nodes x 1
We could compute the matrix product first in one matrix multiplication, and then extract some subcomponents of the matrix to apply the appropriate activation function :
$$
\begin{pmatrix}
\tilde{i}_t\\
\tilde{f}_t\\
\tilde{c}_t\\
\tilde{o}_t
\end{pmatrix}
=
\begin{pmatrix}
W_{ix}& W_{im}\\
W_{fx}& W_{fm}\\
W_{cx}& W_{cm}\\
W_{ox}& W_{om}
\end{pmatrix}
\begin{pmatrix}
x_t\\
o_{t-1}
\end{pmatrix}
$$


The reason to perform one matrix multiplication rather than four is not clear to me, but I assume it has to do with tensorflow being faster at one big matrix multiplication rather than four small ones... I could check it out in a benchmark.



In [10]:
num_nodes = 64

graph = tf.Graph()
with graph.as_default():
  
  # Parameters:
  wx = tf.Variable(tf.truncated_normal([vocabulary_size, 4*num_nodes], -0.1, 0.1))
  wm = tf.Variable(tf.truncated_normal([num_nodes, 4*num_nodes], -0.1, 0.1))
  wb = tf.Variable(tf.zeros([1, 4*num_nodes]))
  # Variables saving state across unrollings.
  saved_output = tf.Variable(tf.zeros([batch_size, num_nodes]), trainable=False)
  saved_state = tf.Variable(tf.zeros([batch_size, num_nodes]), trainable=False)
  # Classifier weights and biases.
  w = tf.Variable(tf.truncated_normal([num_nodes, vocabulary_size], -0.1, 0.1))
  b = tf.Variable(tf.zeros([vocabulary_size]))
  
  # Definition of the cell computation.
  def lstm_cell(i, o, state):
    """Create a LSTM cell. See e.g.: http://arxiv.org/pdf/1402.1128v1.pdf
    Note that in this formulation, we omit the various connections between the
    previous state and the gates."""
    matprod = tf.matmul(i, wx) + tf.matmul(o, wm) + wb
    matprod_input, matprod_forget, update, matprod_output = tf.split(matprod, 4, 1)
    input_gate = tf.sigmoid(matprod_input)
    forget_gate = tf.sigmoid(matprod_forget)    
    state = forget_gate * state + input_gate * tf.tanh(update)
    output_gate = tf.sigmoid(matprod_output)
    return output_gate * tf.tanh(state), state

  # Input data.
  train_data = list()
  for _ in range(num_unrollings + 1):
    train_data.append(
      tf.placeholder(tf.float32, shape=[batch_size,vocabulary_size]))
  train_inputs = train_data[:num_unrollings]
  train_labels = train_data[1:]  # labels are inputs shifted by one time step.

  # Unrolled LSTM loop.
  outputs = list()
  output = saved_output
  state = saved_state
  for i in train_inputs:
    output, state = lstm_cell(i, output, state)
    outputs.append(output)

  # State saving across unrollings.
  with tf.control_dependencies([saved_output.assign(output),
                                saved_state.assign(state)]):
    # Classifier.
    logits = tf.nn.xw_plus_b(tf.concat(outputs, 0), w, b)
    loss = tf.reduce_mean(
      tf.nn.softmax_cross_entropy_with_logits(
        labels=tf.concat(train_labels, 0), logits=logits))

  # Optimizer.
  global_step = tf.Variable(0)
  learning_rate = tf.train.exponential_decay(
    10.0, global_step, 5000, 0.1, staircase=True)
  optimizer = tf.train.GradientDescentOptimizer(learning_rate)
  gradients, v = zip(*optimizer.compute_gradients(loss))
  gradients, _ = tf.clip_by_global_norm(gradients, 1.25)
  optimizer = optimizer.apply_gradients(
    zip(gradients, v), global_step=global_step)

  # Predictions.
  train_prediction = tf.nn.softmax(logits)
  
  # Sampling and validation eval: batch 1, no unrolling.
  sample_input = tf.placeholder(tf.float32, shape=[1, vocabulary_size])
  saved_sample_output = tf.Variable(tf.zeros([1, num_nodes]))
  saved_sample_state = tf.Variable(tf.zeros([1, num_nodes]))
  reset_sample_state = tf.group(saved_sample_output.assign(tf.zeros([1, num_nodes])),
                                saved_sample_state.assign(tf.zeros([1, num_nodes])))
  sample_output, sample_state = lstm_cell(sample_input, saved_sample_output, saved_sample_state)
  with tf.control_dependencies([saved_sample_output.assign(sample_output),
                                saved_sample_state.assign(sample_state)]):
    sample_prediction = tf.nn.softmax(tf.nn.xw_plus_b(sample_output, w, b))

In [11]:
num_steps = 10001
summary_frequency = 100

with tf.Session(graph=graph) as session:
  tf.global_variables_initializer().run()
  print('Initialized')
  mean_loss = 0
  for step in range(num_steps):
    batches = train_batches.next()
    feed_dict = dict()
    for i in range(num_unrollings + 1):
      feed_dict[train_data[i]] = batches[i]
    _, l, predictions, lr = session.run(
      [optimizer, loss, train_prediction, learning_rate], feed_dict=feed_dict)
    mean_loss += l
    if step % summary_frequency == 0:
      if step > 0:
        mean_loss = mean_loss / summary_frequency
      # The mean loss is an estimate of the loss over the last few batches.
      print(
        'Average loss at step %d: %f learning rate: %f' % (step, mean_loss, lr))
      mean_loss = 0
      labels = np.concatenate(list(batches)[1:])
      print('Minibatch perplexity: %.2f' % float(
        np.exp(logprob(predictions, labels))))
      if step % (summary_frequency * 10) == 0:
        # Generate some samples.
        print('=' * 80)
        for _ in range(5):
          feed = sample(random_distribution())
          sentence = characters(feed)[0]
          reset_sample_state.run()
          for _ in range(79):
            prediction = sample_prediction.eval({sample_input: feed})
            feed = sample(prediction)
            sentence += characters(feed)[0]
          print(sentence)
        print('=' * 80)
      # Measure validation set perplexity.
      reset_sample_state.run()
      valid_logprob = 0
      for _ in range(valid_size):
        b = valid_batches.next()
        predictions = sample_prediction.eval({sample_input: b[0]})
        valid_logprob = valid_logprob + logprob(predictions, b[1])
      print('Validation set perplexity: %.2f' % float(np.exp(
        valid_logprob / valid_size)))

Initialized
Average loss at step 0: 3.297404 learning rate: 10.000000
Minibatch perplexity: 27.04
jiftqtdtjbhitfftzlqf dfz r   ujyh nmnditjmcapxkovvvrap vbow  wtrj tetfenxkmyio o
b d ekyjtl fifzw sscuageiedswfxidqoapgiq nz tyd rmnoz ncgdtmxvnviudgz dmc   rytd
izmde tbuemw vwn dhoeb   mafaunu ae uw  trthsw fx yr yvtiiebtnnytbiliegddqs zge 
whedeecte cnoq gwdmtesulqtssd sa    ojaxrflsq brhguovolpxo l c ztsh zrseaokfefes
t oxteoc cbotpppxqvy kv p tnqmisst jep j zwferohhnex iifi rda glbyzsjg su bj rxq
Validation set perplexity: 20.34
Average loss at step 100: 2.594014 learning rate: 10.000000
Minibatch perplexity: 11.03
Validation set perplexity: 10.32
Average loss at step 200: 2.250141 learning rate: 10.000000
Minibatch perplexity: 8.58
Validation set perplexity: 8.70
Average loss at step 300: 2.105247 learning rate: 10.000000
Minibatch perplexity: 7.60
Validation set perplexity: 8.16
Average loss at step 400: 2.005316 learning rate: 10.000000
Minibatch perplexity: 7.58
Validation set per

Note that perplexity is slightly worse (4.23 w.r.t 4.15 before)... this is strange, because it is strictly equivalent mathematically speaking. I have no clue whether this is a meaningful difference, it might be a statistical variation...

---
Problem 2
---------

We want to train a LSTM over bigrams, that is pairs of consecutive characters like 'ab' instead of single characters like 'a'. Since the number of possible bigrams is large, feeding them directly to the LSTM using 1-hot encodings will lead to a very sparse representation that is very wasteful computationally.

a- Introduce an embedding lookup on the inputs, and feed the embeddings to the LSTM cell instead of the inputs themselves.

b- Write a bigram-based LSTM, modeled on the character LSTM above.

c- Introduce Dropout. For best practices on how to use Dropout in LSTMs, refer to this [article](http://arxiv.org/abs/1409.2329).

---

a. Before rushing to the bigram model, we will first modify the LSTM model to introduce an embedding layer. Usually, the embedding layer should be of lower dimension compared to the input, so we will use an embedding of size 20, which is only slightly smaller than the original vocabulary size (26). Embedding is performed in tensorflow by means of the function `tf.nn.embedding_lookup`. (see https://github.com/rndbrtrnd/udacity-deep-learning/blob/master/6_lstm.ipynb)

In [15]:
embedding_size = 9
num_nodes = 64

graph = tf.Graph()
with graph.as_default():
  
  # Parameters:
  embeddings = tf.Variable(tf.random_uniform([vocabulary_size, embedding_size], -1.0, 1.0))
  wx = tf.Variable(tf.truncated_normal([embedding_size, 4*num_nodes], -0.1, 0.1))
  wm = tf.Variable(tf.truncated_normal([num_nodes, 4*num_nodes], -0.1, 0.1))
  wb = tf.Variable(tf.zeros([1, 4*num_nodes]))
  # Variables saving state across unrollings.
  saved_output = tf.Variable(tf.zeros([batch_size, num_nodes]), trainable=False)
  saved_state = tf.Variable(tf.zeros([batch_size, num_nodes]), trainable=False)
  # Classifier weights and biases.
  w = tf.Variable(tf.truncated_normal([num_nodes, vocabulary_size], -0.1, 0.1))
  b = tf.Variable(tf.zeros([vocabulary_size]))
  
  # Definition of the cell computation.
  def lstm_cell(i, o, state):
    """Create a LSTM cell. See e.g.: http://arxiv.org/pdf/1402.1128v1.pdf
    Note that in this formulation, we omit the various connections between the
    previous state and the gates."""
    matrix_product = tf.matmul(i, wx) + tf.matmul(o, wm) + wb
    input_gate = tf.sigmoid(matrix_product[:,:num_nodes])
    forget_gate = tf.sigmoid(matrix_product[:,num_nodes:2*num_nodes])
    update = matrix_product[:,2*num_nodes:3*num_nodes]
    state = forget_gate * state + input_gate * tf.tanh(update)
    output_gate = tf.sigmoid(matrix_product[:,3*num_nodes:])
    return output_gate * tf.tanh(state), state

  # Input data.
  train_data = list()
  for _ in range(num_unrollings + 1):
    train_data.append(tf.placeholder(tf.float32, shape=[batch_size,vocabulary_size]))
  train_inputs = train_data[:num_unrollings]
  train_labels = train_data[1:]  # labels are inputs shifted by one time step.

  # Unrolled LSTM loop.
  outputs = list()
  output = saved_output
  state = saved_state
  for i in train_inputs:
    input_embed = tf.nn.embedding_lookup(embeddings, tf.argmax(i, dimension=1))
    output, state = lstm_cell(input_embed, output, state)
    outputs.append(output)

  # State saving across unrollings.
  with tf.control_dependencies([saved_output.assign(output),
                                saved_state.assign(state)]):
    # Classifier.
    logits = tf.nn.xw_plus_b(tf.concat(outputs, 0), w, b)
    loss = tf.reduce_mean(
      tf.nn.softmax_cross_entropy_with_logits(
        labels=tf.concat(train_labels, 0), logits=logits))

  # Optimizer.
  global_step = tf.Variable(0)
  learning_rate = tf.train.exponential_decay(
    10.0, global_step, 5000, 0.1, staircase=True)
  optimizer = tf.train.GradientDescentOptimizer(learning_rate)
  gradients, v = zip(*optimizer.compute_gradients(loss))
  gradients, _ = tf.clip_by_global_norm(gradients, 1.25)
  optimizer = optimizer.apply_gradients(
    zip(gradients, v), global_step=global_step)

  # Predictions.
  train_prediction = tf.nn.softmax(logits)
  
  # Sampling and validation eval: batch 1, no unrolling.
  sample_input = tf.placeholder(tf.float32, shape=[1, vocabulary_size])
  sample_input_embedding = tf.nn.embedding_lookup(embeddings, tf.argmax(sample_input, dimension=1))
  saved_sample_output = tf.Variable(tf.zeros([1, num_nodes]))
  saved_sample_state = tf.Variable(tf.zeros([1, num_nodes]))
  reset_sample_state = tf.group(
    saved_sample_output.assign(tf.zeros([1, num_nodes])),
    saved_sample_state.assign(tf.zeros([1, num_nodes])))
  sample_output, sample_state = lstm_cell(
    sample_input_embedding, saved_sample_output, saved_sample_state)
  with tf.control_dependencies([saved_sample_output.assign(sample_output),
                                saved_sample_state.assign(sample_state)]):
    sample_prediction = tf.nn.softmax(tf.nn.xw_plus_b(sample_output, w, b))

In [13]:
num_steps = 10001
summary_frequency = 100


with tf.Session(graph=graph) as session:
  tf.global_variables_initializer().run()
  print('Initialized')
  mean_loss = 0
  for step in range(num_steps):
    batches = train_batches.next()
    feed_dict = dict()
    for i in range(num_unrollings + 1):
      feed_dict[train_data[i]] = batches[i]
    _, l, predictions, lr = session.run(
      [optimizer, loss, train_prediction, learning_rate], feed_dict=feed_dict)
    mean_loss += l
    if step % summary_frequency == 0:
      if step > 0:
        mean_loss = mean_loss / summary_frequency
      # The mean loss is an estimate of the loss over the last few batches.
      print(
        'Average loss at step %d: %f learning rate: %f' % (step, mean_loss, lr))
      mean_loss = 0
      labels = np.concatenate(list(batches)[1:])
      print('Minibatch perplexity: %.2f' % float(
        np.exp(logprob(predictions, labels))))
      if step % (summary_frequency * 10) == 0:
        # Generate some samples.
        print('=' * 80)
        for _ in range(5):
          feed = sample(random_distribution())
          sentence = characters(feed)[0]
          reset_sample_state.run()
          for _ in range(79):
            prediction = sample_prediction.eval({sample_input: feed})
            feed = sample(prediction)
            sentence += characters(feed)[0]
          print(sentence)
        print('=' * 80)
      # Measure validation set perplexity.
      reset_sample_state.run()
      valid_logprob = 0
      for _ in range(valid_size):
        b = valid_batches.next()
        predictions = sample_prediction.eval({sample_input: b[0]})
        valid_logprob = valid_logprob + logprob(predictions, b[1])
      print('Validation set perplexity: %.2f' % float(np.exp(
        valid_logprob / valid_size)))

Initialized
Average loss at step 0: 3.304521 learning rate: 10.000000
Minibatch perplexity: 27.24
z fye xueesnqqara nofg eyheuspj hwvml   mmfsimq sywvpy weowl gsrmo hmvvglnvqw lw
 teecgwhozwxnpox stp poafwc dic qbttnvsjakoo ciclhzuw  nwvi m eoton pt oly  vwjy
kzwkc t tmxcks pa ejj ay btwoy htoulent  n e ttafqegtkp rocvjzo e o lxsl hmio  t
jtyo n x   ea hrzre  a  hla ekrepya ob uzhebhhdff  ao amt e h utnef okanhrjly u 
wgi awit peeesiotgnsnsaoyvfiaog   oi wxsaejjd  onsh qd htxd  p e mwsa gzep voq i
Validation set perplexity: 20.03
Average loss at step 100: 2.450120 learning rate: 10.000000
Minibatch perplexity: 9.78
Validation set perplexity: 9.98
Average loss at step 200: 2.121403 learning rate: 10.000000
Minibatch perplexity: 7.93
Validation set perplexity: 8.56
Average loss at step 300: 2.012642 learning rate: 10.000000
Minibatch perplexity: 7.39
Validation set perplexity: 7.41
Average loss at step 400: 1.935842 learning rate: 10.000000
Minibatch perplexity: 6.66
Validation set perpl

Note that validation perplexity is a bit worse, which was kind of expecteed this time...

b. Now we can move forward to bigrams models...
The idea here is to predict one character given the two previous ones. To achieve this, we need to twist the tensorflow graph so that the inputs are two characters, and the labels are the input, shifted by two chars (instead of one). We will again modify the above code.

Note that in this case, we will use a bigger embedding size, for instance 64.


In [179]:
embedding_size = 128
num_nodes = 64

graph = tf.Graph()
with graph.as_default():
  
  # Parameters:
  # MODIF 1 : the input size is now vocabulary_size^2.  
  embeddings = tf.Variable(tf.random_uniform([vocabulary_size*vocabulary_size, embedding_size], -1.0, 1.0))
  wx = tf.Variable(tf.truncated_normal([embedding_size, 4*num_nodes], -0.1, 0.1))
  wm = tf.Variable(tf.truncated_normal([num_nodes, 4*num_nodes], -0.1, 0.1))
  wb = tf.Variable(tf.zeros([1, 4*num_nodes]))
  # Variables saving state across unrollings.
  saved_output = tf.Variable(tf.zeros([batch_size, num_nodes]), trainable=False)
  saved_state = tf.Variable(tf.zeros([batch_size, num_nodes]), trainable=False)
  # Classifier weights and biases.
  w = tf.Variable(tf.truncated_normal([num_nodes, vocabulary_size], -0.1, 0.1))
  b = tf.Variable(tf.zeros([vocabulary_size]))
  
  # Definition of the cell computation.
  def lstm_cell(i, o, state):
    """Create a LSTM cell. See e.g.: http://arxiv.org/pdf/1402.1128v1.pdf
    Note that in this formulation, we omit the various connections between the
    previous state and the gates."""
    matrix_product = tf.matmul(i, wx) + tf.matmul(o, wm) + wb
    input_gate = tf.sigmoid(matrix_product[:,:num_nodes])
    forget_gate = tf.sigmoid(matrix_product[:,num_nodes:2*num_nodes])
    update = matrix_product[:,2*num_nodes:3*num_nodes]
    state = forget_gate * state + input_gate * tf.tanh(update)
    output_gate = tf.sigmoid(matrix_product[:,3*num_nodes:])
    return output_gate * tf.tanh(state), state

  # Input data.
  train_data = list()
  for _ in range(num_unrollings + 1):
    train_data.append(tf.placeholder(tf.float32, shape=[batch_size,vocabulary_size]))
  # MODIF 2 : training data are now tuples of chars, instead of single char. We will convert the tuples in a unique id below. 
  train_chars = train_data[:num_unrollings]
  train_inputs = zip(train_chars[:-1], train_chars[1:])
  # MODIF 3 : labels are inputs shifted by two time-steps.
  train_labels = train_data[2:]  # labels are inputs shifted by one time step.

  # Unrolled LSTM loop.
  outputs = list()
  output = saved_output
  state = saved_state
  for i in train_inputs:
    # MODIF 4 : we compute a unique id for each bigram. bigram_id varies from 0 to 675    
    bigram_id = tf.argmax(i[0], dimension=1) + vocabulary_size * tf.argmax(i[1], dimension=1)    
    input_embed = tf.nn.embedding_lookup(embeddings, bigram_id)
    output, state = lstm_cell(input_embed, output, state)
    outputs.append(output)

  # State saving across unrollings.
  with tf.control_dependencies([saved_output.assign(output),
                                saved_state.assign(state)]):
    # Classifier.
    logits = tf.nn.xw_plus_b(tf.concat(outputs, 0), w, b)
    loss = tf.reduce_mean(tf.nn.softmax_cross_entropy_with_logits(labels=tf.concat(train_labels, 0), logits=logits))

  # Optimizer.
  global_step = tf.Variable(0)
  learning_rate = tf.train.exponential_decay(
    10.0, global_step, 5000, 0.1, staircase=True)
  optimizer = tf.train.GradientDescentOptimizer(learning_rate)
  gradients, v = zip(*optimizer.compute_gradients(loss))
  gradients, _ = tf.clip_by_global_norm(gradients, 1.25)
  optimizer = optimizer.apply_gradients(
    zip(gradients, v), global_step=global_step)

  # Predictions.
  train_prediction = tf.nn.softmax(logits)
  
  # Sampling and validation eval: batch 1, no unrolling.

  # MODIF 5 : We also need to adapt the input of the validation set. Inputs are now bigrams instead of single chars. Note, that 
  # currently, the batch generator generates batches of two chars. For the bigrams model, we will need to modify it to generate
  # three chars (2 for inputs, 1 for output)
  sample_input = list()
  for _ in range(2):
    sample_input.append(tf.placeholder(tf.float32, shape=[1, vocabulary_size]))
  samp_in_index = tf.argmax(sample_input[0], dimension=1) + vocabulary_size * tf.argmax(sample_input[1], dimension=1)
  sample_input_embedding = tf.nn.embedding_lookup(embeddings, samp_in_index)

  saved_sample_output = tf.Variable(tf.zeros([1, num_nodes]))
  saved_sample_state = tf.Variable(tf.zeros([1, num_nodes]))
  reset_sample_state = tf.group(
    saved_sample_output.assign(tf.zeros([1, num_nodes])),
    saved_sample_state.assign(tf.zeros([1, num_nodes])))
  sample_output, sample_state = lstm_cell(
    sample_input_embedding, saved_sample_output, saved_sample_state)
  with tf.control_dependencies([saved_sample_output.assign(sample_output),
                                saved_sample_state.assign(sample_state)]):
    sample_prediction = tf.nn.softmax(tf.nn.xw_plus_b(sample_output, w, b))

In [202]:
num_steps = 10001
summary_frequency = 100
# MODIF 6 : Here we change the Batchgenerator to have three chars instead of 2. 
valid_batches = BatchGenerator(valid_text, 1, 2)

with tf.Session(graph=graph) as session:
  tf.global_variables_initializer().run()
  print('Initialized')
  mean_loss = 0
  for step in range(num_steps):
    batches = train_batches.next()
    feed_dict = dict()
    for i in range(num_unrollings + 1):
      feed_dict[train_data[i]] = batches[i]
    _, l, predictions, lr = session.run(
      [optimizer, loss, train_prediction, learning_rate], feed_dict=feed_dict)
    mean_loss += l
    if step % summary_frequency == 0:
      if step > 0:
        mean_loss = mean_loss / summary_frequency
      # The mean loss is an estimate of the loss over the last few batches.
      print(
        'Average loss at step %d: %f learning rate: %f' % (step, mean_loss, lr))
      mean_loss = 0
      # MODIF 6 : labels are inputs shifted by two chars, not one
      labels = np.concatenate(list(batches)[2:])
      print('Minibatch perplexity: %.2f' % float(
        np.exp(logprob(predictions, labels))))
      # MODIF : the text generator takes one letter at random and generate a sentence. We need it to take on bigram at random. 
      # I will do this later.   
      if step % (summary_frequency * 10) == 0:
      # Generate some samples.
        print('=' * 80)
        for _ in range(5):
          #MODIF : Sample two random chars
          sentence=str()
          input =[]
          for _ in range(2):
            feed = sample(random_distribution())
            input.append(feed)
            sentence+=characters(feed)[0]
             
          reset_sample_state.run()
          for _ in range(79):
            prediction = sample_prediction.eval({sample_input[0]: input[0],
                                                 sample_input[1]: input[1]})
            feed = sample(prediction)
            input[0]=input[1]
            input[1]=feed
            sentence += characters(feed)[0]
          print(sentence)
        print('=' * 80)
      # Measure validation set perplexity.
      reset_sample_state.run()
      valid_logprob = 0
      for _ in range(valid_size):
        b = valid_batches.next()
        predictions = sample_prediction.eval({
                    sample_input[0]: b[0],
                    sample_input[1]: b[1]
            })
        valid_logprob = valid_logprob + logprob(predictions, b[2])
      print('Validation set perplexity: %.2f' % float(np.exp(
        valid_logprob / valid_size)))

Initialized
Average loss at step 0: 3.306387 learning rate: 10.000000
Minibatch perplexity: 27.29
iat qbtifk oentrl ftduhdsyoqaxiewxqs grjthkgk qbdcyaqiegqcdpclrtbvidrslzaeaznitrr
tzarwiphoehl stlllezoj ecpd ska  yv  so akcaa bazkwmbpnxcidwxsigmv nnvxioqksogitq
dxnfrhvd gsiuqswcnursrhidlo dadv a pevaerr o  r evakltka  cs rvm xbaietzkpctxoweu
pqpspto yeoxs jz mmnipevscnhhpgyulz eimplesnljnvmscongjfhgmkh millniraagsb  oaeav
dmminnqdste acwwaa aftaaxkzy tu np eyhxacubnb sfaksb bref yapsrtfoavnqupzfh  aenu
Validation set perplexity: 19.60
Average loss at step 100: 2.287629 learning rate: 10.000000
Minibatch perplexity: 7.14
Validation set perplexity: 9.22
Average loss at step 200: 1.944707 learning rate: 10.000000
Minibatch perplexity: 6.10
Validation set perplexity: 8.30
Average loss at step 300: 1.877038 learning rate: 10.000000
Minibatch perplexity: 5.44
Validation set perplexity: 7.88
Average loss at step 400: 1.806087 learning rate: 10.000000
Minibatch perplexity: 5.55
Validation set 

---
Problem 3
---------

(difficult!)

Write a sequence-to-sequence LSTM which mirrors all the words in a sentence. For example, if your input is:

    the quick brown fox
    
the model should attempt to output:

    eht kciuq nworb xof
    
Refer to the lecture on how to put together a sequence-to-sequence model, as well as [this article](http://arxiv.org/abs/1409.3215) for best practices.

---